In [60]:
# Loading packages
using CSV, DataFrames, Gurobi, JuMP, GLPK, Ipopt

In [66]:
# read data
df_simplified = CSV.read("Input/simplified_train_with_index_for_Zeki.csv", DataFrame)
# change the type of the column "next_wap" to float
df_simplified[!, :next_wap] = convert(Array{Float64,1}, df_simplified[!, :next_wap])
# change the type of the column "next_idex" to float
df_simplified[!, :next_index] = convert(Array{Float64,1}, df_simplified[!, :next_index])
df_simplified

Row,stock_id,date_id,seconds_in_bucket,wap,target,time_id,row_id,weight,weighted_wap,index,next_wap,next_index
,Int64,Int64,Int64,Float64,Float64,Int64,String15,Float64,Float64,Float64,Float64,Float64
1,0,451,490,1.00096,1.82033,24854,451_490_0,0.004,0.00400382,1.00108,0.0,0.0
2,1,451,490,1.00326,5.78046,24854,451_490_1,0.00099987,0.00100313,1.00108,0.0,0.0
3,2,451,490,0.99922,4.99964,24854,451_490_2,0.00200041,0.00199885,1.00108,0.0,0.0
4,3,451,490,1.00087,0.369549,24854,451_490_3,0.00599891,0.00600411,1.00108,0.0,0.0
5,4,451,490,1.00009,-1.27971,24854,451_490_4,0.00400068,0.00400104,1.00108,0.0,0.0
6,5,451,490,1.00187,-7.28011,24854,451_490_5,0.00399949,0.00400696,1.00108,0.0,0.0
7,6,451,490,0.997284,-8.55982,24854,451_490_6,0.00200014,0.00199471,1.00108,0.0,0.0
8,7,451,490,1.00021,0.400543,24854,451_490_7,0.0059992,0.00600044,1.00108,0.0,0.0
9,8,451,490,0.99927,-2.83003,24854,451_490_8,0.00600029,0.00599591,1.00108,0.0,0.0


In [4]:
weights = [0.004,0.00099987 ,0.00200041 ,0.00599891 ,0.00400068 ,0.00399949
 ,0.00200014 ,0.0059992  ,0.00600029 ,0.00200012 ,0.00200072 ,0.00800012
 ,0.00600039 ,0.00200007 ,0.0080014  ,0.00600051 ,0.00199993 ,0.0059999
 ,0.00400013 ,0.00199974 ,0.00399933 ,0.00099993 ,0.00599955 ,0.00399971
 ,0.00199876 ,0.00200008 ,0.00400097 ,0.0020008  ,0.00400021 ,0.00400005
 ,0.00100001 ,0.00100016 ,0.00199924 ,0.00199993 ,0.00599995 ,0.00399917
 ,0.00400017 ,0.00399906 ,0.00599977 ,0.00200015 ,0.0020007  ,0.03999955
 ,0.0020001  ,0.00199981 ,0.00399968 ,0.03999983 ,0.00200033 ,0.00099959
 ,0.00600002 ,0.00399944 ,0.00399979 ,0.00600017 ,0.00099921 ,0.00399993
 ,0.00399936 ,0.0019985  ,0.00599995 ,0.00400028 ,0.00599981 ,0.00400045
 ,0.00600029 ,0.00399988 ,0.00200012 ,0.00100003 ,0.00200041 ,0.00400043
 ,0.00200011 ,0.00799989 ,0.00400056 ,0.00400011 ,0.00199956 ,0.00399921
 ,0.00599994 ,0.00199992 ,0.00399979 ,0.00400047 ,0.00200035 ,0.00399996
 ,0.00400003 ,0.00399989 ,0.00100029 ,0.00199997 ,0.00200019 ,0.00799969
 ,0.02000066 ,0.00400029 ,0.00600019 ,0.00200024 ,0.02       ,0.00199965
 ,0.00199968 ,0.00599962 ,0.00399987 ,0.00199997 ,0.00099953 ,0.02000033
 ,0.00600029 ,0.00099946 ,0.00200062 ,0.00399946 ,0.00100021 ,0.00199986
 ,0.00600047 ,0.00600071 ,0.00399889 ,0.00600028 ,0.0010003  ,0.00199986
 ,0.00400008 ,0.00600071 ,0.00600016 ,0.00100013 ,0.04000011 ,0.00599947
 ,0.00200012 ,0.00400061 ,0.00199956 ,0.00199961 ,0.00599985 ,0.00200055
 ,0.00200083 ,0.00399993 ,0.00599961 ,0.00600028 ,0.00200005 ,0.00199999
 ,0.0080003  ,0.00600021 ,0.00400044 ,0.00199994 ,0.00599988 ,0.00200002
 ,0.00399908 ,0.00600099 ,0.00200021 ,0.00399981 ,0.00100042 ,0.00399994
 ,0.00199993 ,0.0039995  ,0.00800063 ,0.00600055 ,0.00799953 ,0.00200005
 ,0.00400064 ,0.00199988 ,0.00099981 ,0.00399912 ,0.00399992 ,0.00399974
 ,0.00599995 ,0.00800022 ,0.00400055 ,0.00099968 ,0.00099956 ,0.00200017
 ,0.00599973 ,0.00400059 ,0.00100004 ,0.00199961 ,0.00599971 ,0.00400006
 ,0.00599987 ,0.00800007 ,0.00200054 ,0.00200102 ,0.00400114 ,0.00199923
 ,0.0399987  ,0.00199969 ,0.00199982 ,0.00399976 ,0.00199995 ,0.00200001
 ,0.00599982 ,0.02000055 ,0.0039996  ,0.00200022 ,0.00600002 ,0.01999984
 ,0.00100015 ,0.00200021 ,0.0059996  ,0.00400055 ,0.00599988 ,0.00400022
 ,0.00399931 ,0.0040002  ,0.0039998  ,0.002      ,0.00399975 ,0.04000041
 ,0.00200037 ,0.00799998 ,0.00199977 ,0.00400093 ,0.00099996 ,0.00400022
 ,0.00599947 ,0.00400014];

In [87]:
function index_calculate(T, S, I, W)

    # define the number of stock
    n = 200

    # define the model
    model = Model(Ipopt.Optimizer)

    # define the variables
    @variable(model, SP[1:n])
    @variable(model, IP)

    # define the constraints
    @constraint(model, IP == sum(W .* SP))
    # @constraint(model, [i=1:n], SP[i] == (T[i]/10000 + IP/I) * S[i])
    @constraint(model, [i=1:n], T[i] == (SP[i]/S[i] - IP/I) * 10000)

    # solve the model
    optimize!(model)

    # extract the variables
    SP = value.(SP)
    IP = value.(IP)

    # return the results
    return SP, IP
end

index_calculate (generic function with 1 method)

In [82]:
# function index_calculate(T, S, I, W)

#     # define the number of stock
#     n = 200

#     # define the model
#     # model = Model(Gurobi.Optimizer)
#     model = Model(GLPK.Optimizer)

#     # define the variables
#     @variable(model, SP[1:n])
#     @variable(model, IP)

#     # define the objective function
#     @objective(model, Min, 0)

#     # define the constraints
#     @constraint(model, IP == sum(W .* SP))
#     # @constraint(model, [i=1:n], SP[i] == (T[i]/10000 + IP/I) * S[i])
#     @constraint(model, [i=1:n], T[i] == (SP[i]/S[i] - IP/I) * 10000)

#     # solve the model
#     optimize!(model)

#     # extract the variables
#     SP = value.(SP)
#     IP = value.(IP)

#     # return the results
#     return SP, IP
# end

index_calculate (generic function with 1 method)

In [88]:
for d in 0:29
    for s in 0:5
        # extract the data
        target = df_simplified[d*200*6 + s*200 + 1 : d*200*6 + s*200 + 200, :target]
        stock_price = df_simplified[d*200*6 + s*200 + 1 : d*200*6 + s*200 + 200, :wap]
        index = df_simplified[d*200*6 + s*200 + 1 : d*200*6 + s*200 + 200, :index][1]
        # calculate the index
        next_stock_price, next_index = index_calculate(target, stock_price, index, weights)
        # save the results
        df_simplified[d*200*6 + s*200 + 1 : d*200*6 + s*200 + 200, :next_wap] .= coalesce.(next_stock_price, missing)
        df_simplified[d*200*6 + s*200 + 1 : d*200*6 + s*200 + 200, :next_index] .= coalesce(next_index, missing)
    end
end

In [89]:
df_simplified

Row,stock_id,date_id,seconds_in_bucket,wap,target,time_id,row_id,weight,weighted_wap,index,next_wap,next_index
,Int64,Int64,Int64,Float64,Float64,Int64,String15,Float64,Float64,Float64,Float64,Float64
1,0,451,490,1.00096,1.82033,24854,451_490_0,0.004,0.00400382,1.00108,180.531,180.553
2,1,451,490,1.00326,5.78046,24854,451_490_1,0.00099987,0.00100313,1.00108,180.947,180.553
3,2,451,490,0.99922,4.99964,24854,451_490_2,0.00200041,0.00199885,1.00108,180.219,180.553
4,3,451,490,1.00087,0.369549,24854,451_490_3,0.00599891,0.00600411,1.00108,180.515,180.553
5,4,451,490,1.00009,-1.27971,24854,451_490_4,0.00400068,0.00400104,1.00108,180.375,180.553
6,5,451,490,1.00187,-7.28011,24854,451_490_5,0.00399949,0.00400696,1.00108,180.695,180.553
7,6,451,490,0.997284,-8.55982,24854,451_490_6,0.00200014,0.00199471,1.00108,179.868,180.553
8,7,451,490,1.00021,0.400543,24854,451_490_7,0.0059992,0.00600044,1.00108,180.396,180.553
9,8,451,490,0.99927,-2.83003,24854,451_490_8,0.00600029,0.00599591,1.00108,180.227,180.553
